# Purpose of this notebook  
Create new features from existing ones using statistical analysis then compared it with random forest feature selection to see how the new feature is getting ranked.<br>
Then we used our features (though they were ranked lowest) to create our model. I've used linear regression with r2 score to determine the outcome .<br>

Creating new feature - How much effective is it ? What are the risks involved ?

In [1056]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [1057]:
pd.pandas.set_option("display.max_rows",None)
pd.pandas.set_option("display.max_columns",None)

In [1058]:
df=pd.read_csv('../input/life-expectancy-who/Life Expectancy Data.csv')

In [1059]:
df.head(5) 

In [1060]:
df.columns

Renaming columns as few contains white spaces which makes column names longer & it gets splitted into a 
newline making it difficult to read

In [1061]:
df.rename(columns={'Life expectancy ':'Life_Expectancy','Adult Mortality':'Adult_Mortality','infant deaths':'Infant_Deaths',
                   'Hepatitis B':'Hepatitis_B','percentage expenditure':'Percentage_Expenditure','Measles ':'Measles',
                   ' BMI ':'BMI','under-five deaths ':'Under-Five_Deaths','Total expenditure':'Total_Expenditure',
                  'Diphtheria ':'Diphtheria',' HIV/AIDS':'HIV_AIDS',' thinness  1-19 years':'Thinness_1-19_Years',
                  ' thinness 5-9 years':'Thinness_5-9_Years','Income composition of resources':'ICR'},inplace=True)
df.columns

In [1062]:
df.info()

In [1063]:
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'missing_value %': percent_missing})
missing_value_df.sort_values('missing_value %',ascending=False, inplace=True)
missing_value_df

In [1064]:
percent_missing = (df == 0).astype(int).sum(axis=0) * 100 / len(df)
zero_value_df = pd.DataFrame({'Zero_value %': percent_missing})
zero_value_df.sort_values('Zero_value %',ascending=False, inplace=True)
zero_value_df

In [1065]:
plt.figure(figsize=(15,10))
sns.heatmap(df.isnull())

before applying correlation we need to check skewness and kurtosis of data<br>


skewness == 0 : normally distributed , can use pearson correation (assuming homogeneity exist)<br>
skewness > 0 : more weight in the left tail of the distribution<br>
skewness < 0 : more weight in the right tail of the distribution<br>

if skewness != 0 we use 
1. kendall : Tau correlation coefficient
2. spearman : Rank correlation

Skewness differentiates extreme values in one versus the other tail <br>
Kurtosis measures extreme values in either tail. <br>

Large kurtsis (+ve/-ve) : five or more standard deviations from the mean , tail data exceeds normal distribution <br>
Low kurtosis : skinny , less extreme than tails of normal distribution


In [1066]:
stats = df.describe()
stats.loc['median'] = df.median().tolist()
stats.loc['skew'] = df.skew().tolist()
stats.loc['kurt'] = df.kurtosis().tolist()
stats_df=pd.DataFrame(stats).reindex(['count','mean','std','median','50%','skew','kurt','min','max','25%','75%'])
stats_df 

**How close Mean & Std is to each other? -** What transformation we need to perform <br>
**How much of it lack symmetry ? - skew** <br>
    symmetric - mean = median = mode = 0     ====> (-0.5,+0.5) <br>
    +ve/right skewed - mean > median > mode  ====> (+0.5,+1)   <br>
    -ve/left skewed - mode > median > mean   ====> (-1,-0.5)   <br>
**Heavy tailed or light tailed ? - kurt** <br>
    kurt > 0 - numbers located in tails , high probability of extreme events <br>
    kurt < 0 - numbers located in high proximity to mean,low probability of extreme events <br>
    -1 > kurt > +1 - sizable departure from normality<br>
    
When data skewed, the tail region may behave as an outlier for the statistical model, and outliers unsympathetically affect the model’s performance especially regression-based models. Some statistical models are hardy to outliers like Tree-based models, but it will limit the possibility to try other models. So there is a necessity to transform the skewed data to close enough to a Normal distribution.

In [1067]:
corr=df.corr(method='spearman',min_periods=10)
plt.figure(figsize=(15,10))
sns.heatmap(data=corr,cmap="RdBu_r",annot=True,vmin=-1,vmax=1)

In [1068]:
corr=df.corr(method='kendall')
plt.figure(figsize=(15,10))
sns.heatmap(data=corr,cmap="RdBu_r",annot=True,vmin=-1,vmax=1)

**+ve correlation (> 50%) -** <br>
[(Infant_Deaths,Under-Five_Deaths),<br>
(GDP,Percentage_Expenditure),<br>
(Diphtheria,Polio - Hepatitis_B),<br>
(ICR,Schooling - Alcohol,BMI),<br>
(Thinness_1-19_Years,Thinness_5-9_Years)]<br>
**-ve correlation (< 50%) -** <br>
[(ICR,Schooling-GDP,BMI,Infant_Deaths,HIV_AIDS,Under-Five_Deaths),<br>
(Thinness_1-19_Years,Thinness_5-9_Years - ICR,Schooling,Alcohol,BMI),<br>
(Polio,Diphtheria,GDP -HIV_AIDS)]<br>
**correlation with Life_Expectancy** - <br>
[Schooling,ICR (+ve = greater than 50%)] , <br>
[HIV_AIDS,Adult_Mortality(-ve = lesser than 50%)] <br>
[GDP,Diphtheria,Polio,BMI(+ve = 0-50%)] <br>
[Thinness_1-19_Years,Thinness_5-9_Years,Under-Five_Deaths,Infant_Deaths (-ve = 0-50% )]

In [1069]:
col=['Life_Expectancy', 'Adult_Mortality', 'Infant_Deaths', 'Alcohol', 'Percentage_Expenditure', 
     'Hepatitis_B','Measles', 'BMI', 'Under-Five_Deaths', 'Polio', 'Total_Expenditure','Diphtheria', 
     'HIV_AIDS', 'GDP', 'Population', 'Thinness_1-19_Years','Thinness_5-9_Years', 'ICR', 'Schooling']

dic={}
#column index mapping for subplot
for i in range(len(col)):
    dic[col[i]]=i+1
# Detect outliers in each variable using box plots.
plt.figure(figsize=(20,30))

for key,val in dic.items():
    plt.subplot(5,5,val)
    sns.histplot(x=df[key],data=df,bins=10,kde=True)
    plt.xlabel(key)
    plt.title(key+' Distribution')

plt.show()

In [1070]:
sns.catplot(x='Status',y='Year',kind='violin',data=df)
plt.figure(figsize=(60,40))
plt.show()

In [1071]:
# from sklearn import preprocessing
# label_encoder = preprocessing.LabelEncoder()
# df['Country']= label_encoder.fit_transform(df['Country']) 
# df['Status']= label_encoder.fit_transform(df['Status']) 

In [1072]:
from sklearn.impute import KNNImputer 
#uses nearest neighbour concept

In [1073]:
imputer=KNNImputer(n_neighbors=10)
#we're using fit_transform so we need a new dataframe to avoid overwrite
df_new=df[df.select_dtypes(include=['int64','float64']).columns]
df_KNN=pd.DataFrame(imputer.fit_transform(df_new))

In [1074]:
df_KNN.columns=['Year', 'Life_Expectancy', 'Adult_Mortality',
       'Infant_Deaths', 'Alcohol', 'Percentage_Expenditure', 'Hepatitis_B',
       'Measles', 'BMI', 'Under-Five_Deaths', 'Polio', 'Total_Expenditure',
       'Diphtheria', 'HIV_AIDS', 'GDP', 'Population', 'Thinness_1-19_Years',
       'Thinness_5-9_Years', 'ICR', 'Schooling']
df_KNN_std_=df_KNN[['Total_Expenditure', 'Population', 'ICR','BMI','Schooling',
                  'Life_Expectancy', 'Adult_Mortality', 'Alcohol', 'Hepatitis_B',
                    'Polio','Diphtheria', 'GDP', 'Thinness_1-19_Years',
                   'Thinness_5-9_Years']].std()
df_KNN_std_

1. central tendency based imputation (+ randomness)
2. random sample imputation 
3. median value imputation

Population 	    -       22.191967<br>
Hepatitis_B 	  -     18.822328<br>
GDP 	         -      15.248468<br>
Total_Expenditure 	-   7.692308<br>
Alcohol 	   -        6.603131<br>
ICR 	    -           5.684139<br>
Schooling 	   -        5.547992<br>
Thinness_5-9_Years 	-   1.157250<br>
Thinness_1-19_Years -   1.157250<br>
BMI 	         -      1.157250<br>
Polio 	       -        0.646698<br>
Diphtheria 	   -        0.646698<br>
Life_Expectancy 	-   0.340368<br>
Adult_Mortality 	-   0.340368<br>

In [1075]:
def central_tendency_imputer(data):
    for col in data.columns.values:
        mean_=data[col].mean()
        median_=data[col].median()
        std_=data[col].std()
        is_null_=data[col].isnull().sum()
        rand_=np.random.choice([mean_- std_,mean_ + std_,is_null_])
        data[col]=data[col].fillna(rand_)
    return data

In [1076]:
def random_sample_imputer(data):
    for col in data.columns.values:
        median_=data[col].median()
        rand_s = data[col].dropna().sample(data[col].isnull().sum(),random_state=0)
        rand_s.index=data[data[col].isnull()].index
        data[col]=data[col].fillna(rand_s)
    return data

In [1077]:
data_ct=df[['Total_Expenditure', 'Population', 'ICR','BMI','Schooling']]
df_CT=central_tendency_imputer(data_ct)
#df_CT=df_CT.drop(['Total_Expenditure', 'Population', 'ICR','BMI','Schooling'],axis=1)
df_CT_std_=df_CT.std()
df_CT_std_

In [1078]:
data_rs=df[['Life_Expectancy', 'Adult_Mortality', 'Alcohol', 'Hepatitis_B',
        'Polio','Diphtheria', 'GDP', 'Thinness_1-19_Years',
       'Thinness_5-9_Years' ]]
df_RS=random_sample_imputer(data_rs)
df_RS_std_=df_RS.std()
df_RS_std_

In [1079]:
df_std_=df[['Total_Expenditure', 'Population', 'ICR','BMI','Schooling',
                  'Life_Expectancy', 'Adult_Mortality', 'Alcohol', 'Hepatitis_B',
                    'Polio','Diphtheria', 'GDP', 'Thinness_1-19_Years',
                   'Thinness_5-9_Years']].std()
df_mean_=df[['Total_Expenditure', 'Population', 'ICR','BMI','Schooling',
                  'Life_Expectancy', 'Adult_Mortality', 'Alcohol', 'Hepatitis_B',
                    'Polio','Diphtheria', 'GDP', 'Thinness_1-19_Years',
                   'Thinness_5-9_Years']].mean()


In [1080]:
df_med_=df[['Total_Expenditure','Population','ICR','BMI','Schooling',
                  'Life_Expectancy', 'Adult_Mortality', 'Alcohol', 'Hepatitis_B',
                    'Polio','Diphtheria', 'GDP', 'Thinness_1-19_Years',
                   'Thinness_5-9_Years']]

for col in df_med_:
    median_=df_med_[col].median()
    df_med_[col]=df_med_[col].fillna(median_)

df_med_std_=df_med_.std()
df_med_.isnull().sum()

In [1081]:
df_manual_std_=pd.concat([df_CT_std_,df_RS_std_],axis=0)
df_std_comp=pd.concat([df_mean_,df_std_,df_KNN_std_,df_manual_std_,df_med_std_],axis=1)
df_std_comp.columns=['Mean','STD','KNN','Manual','Median']
df_std_comp

Replacing null value with Median is giving the bwst result , followed by KNN Imputer . It tells us there are outliers & values are clustered around Median <br>

High/Low std = Low standard deviation means data are clustered around the mean, and high standard deviation indicates data are more spread out<br>

Relation between mean vs std tells us about the outliers too. i.e distribution of data points from max & min<br>

In [1082]:
# Replacing nan values with median of that feature , as it's the best method so far
median=df.median()
df=df.fillna(median)
df.isnull().sum()

Before removing outliers , let's check distributions of other features too 

In [1083]:
col=['Year','Life_Expectancy', 'Adult_Mortality','Infant_Deaths', 'Alcohol', 'Percentage_Expenditure', 
     'Hepatitis_B','Measles', 'BMI', 'Under-Five_Deaths', 'Polio', 'Total_Expenditure',
     'Diphtheria', 'HIV_AIDS', 'GDP', 'Population', 'Thinness_1-19_Years','Thinness_5-9_Years', 
     'ICR', 'Schooling']
dic={}
#map an integer to each column value, which will be used in subplot selection
for i in range(len(col)):
    dic[col[i]]=i+1

    
plt.figure(figsize=(20,30))

for key,val in dic.items():
    plt.subplot(5,5,val)
    sns.boxplot(x=df[key],data=df,boxprops=dict(alpha=0.2))
    plt.xlabel(key)
    plt.title(key+' Distribution')

plt.show()

Feature Transformation :
Infant death,Adult mortality ,Measles -> find % of total population affected per country per year
we can use standardization to scale thing down<br>

compare mortality rate for Hepatitis_B,Measles,Diphtheria,HIV_AIDS using ANOVA <br>

We have less number of records so we just can't drop the outliers , we have to impute them to an acceptable range <br>


As we have seen in previous histogram + kde plot except(Schooling & Total_Expenditure) none of them follow Normal distribution so we're trying these transformation to make them as close as Normal Distribution - Thanks to [Krish Naik Feature Engineering](https://github.com/krishnaik06/Feature-Engineering-Live-sessions/blob/master/Feature%20Engineering-%20Normalization%20And%20Standardization-%20Day%205.ipynb)

1. logarithmic transformation
2. reciprocal transformation
3. square root transformation
4. exponential transformation (more general, you can use any exponent)
5. boxcox transformation (intentionally commented it as it modifies datafram , we've to remove that column later)

As none of them follow normal distribution we can't trust one-way ANOVA test (though it can tolerate data that is non-normal (skewed or kurtotic distributions) with only a small effect on the Type I error rate

In [1084]:
# from scipy.stats import f_oneway
# f_oneway(df['Hepatitis_B'],df['Measles'],df['Diphtheria'],df['HIV_AIDS'])
#F_onewayResult(statistic=90.43876694736144, pvalue=7.409267726887965e-58)

# **Now the most interesting part creating new features - How & Why ??** <br>
**Disease_Death** - Disease causing death , it doesn't matter what disease the effect is same for all of them so we can combine them into one feature , upon carefully reading Measles & HIV_AIDS are the 2 disease we're interested on<br>
1. According to data description sample was collected per 100 population , so we scale down to 100<br>
2. Still the value is high , so apply reciprocal transformation (since log is too small for that)<br>

**Mortality_Rate** - We have 3 potential candidate here - Adult_Mortality,Infant_Deaths,Under-Five_Deaths
Here the 1st question->  If infant(0-1) did cover under Under-Five_Deaths category why to put it on 1st place ?? <br>

Ans -> we don't know for sure , may be keeping track of birth-cirtificate is easier than death certificate but how do we deal with it ? <br>

Ans -> Upon looking at barplot we see a striking similarity between Infant_Deaths,Under-Five_Deaths so we can take any one, so for proper distribution = Adult_Mortality+Infant_Deaths-Under-Five_Deaths , then we do a scale down to per 100 population<br>

what if value becomes -ve so we take abs() -> till high do a square root (reduce a large amount ) -> now we need to reduce a small amount carefully (upon checking boxplot) we use log trasformation<br>

**Vaccination_Immunity** - we have Polio,Hepatitis_B,Diphtheria these are disease from mostly developing country where government expenditure is a big factor for eradicationg these viruses <br>

So we calculate how effeciently money is being used to vaccinate - <br> 
Numerator = vaccination rate , Denominator = avg. expenditure = (Gov. + personal) / 2 -> we smoothen the distribution by slightly shifting the tails using log transformation<br>


**+ve correlation (> 50%) -** <br>
[(Infant_Deaths,Under-Five_Deaths),<br>
(GDP,Percentage_Expenditure),<br>
(Diphtheria,Polio - Hepatitis_B),<br>
(ICR,Schooling - Alcohol,BMI),<br>
(Thinness_1-19_Years,Thinness_5-9_Years)]<br>
**-ve correlation (< 50%) -** <br>
[(ICR,Schooling-GDP,BMI,Infant_Deaths,HIV_AIDS,Under-Five_Deaths),<br>
(Thinness_1-19_Years,Thinness_5-9_Years - ICR,Schooling,Alcohol,BMI),<br>
(Polio,Diphtheria,GDP -HIV_AIDS)]<br>
**correlation with Life_Expectancy** - <br>
[Schooling,ICR (+ve = greater than 50%)] , <br>
[HIV_AIDS,Adult_Mortality(-ve = lesser than 50%)] <br>
[GDP,Diphtheria,Polio,BMI(+ve = 0-50%)] <br>
[Thinness_1-19_Years,Thinness_5-9_Years,Under-Five_Deaths,Infant_Deaths (-ve = 0-50% )]<br>

from above list - <br>
1. GDP,Percentage_Expenditure(+ve)- > Percentage_Expenditure taken as representative feature
2. Polio,Diphtheria,GDP - HIV_AIDS(-ve)-> HIV_AIDS has been used as indicator of death & Polio,Diphtheria immunity from death/close to death (vaccination)
3. ICR,Schooling - GDP,BMI,Infant_Deaths,HIV_AIDS,Under-Five_Deaths(-ve) - Schooling is an important factor so how a counytry is providing education to it's people that's important , here other features have been aggregated to new representative feature
4. Thinness_1-19_Years,Thinness_5-9_Years(+ve) - combined to a single representative feature
5. Infant_Deaths & Under-Five_Deaths = Mortality Rate
6. Thinness_1-19_Years,Thinness_5-9_Years - ICR,Schooling,Alcohol,BMI(-ve) - combined Alcohol,BMI to a new feature 'Changed_Life_Expectancy' on which we'll train our model 

In [1085]:
df['Disease_Death']=(df['Measles']+df['HIV_AIDS'])/10
df['Disease_Death']=1/df['Disease_Death']
df['Mortality_Rate']=df['Adult_Mortality']+df['Infant_Deaths']-df['Under-Five_Deaths']/10
df['Mortality_Rate']=np.log((df['Mortality_Rate'].abs())**(1/2))
df['Vaccination_Immunity']=np.log(2*(df['Polio']+df['Hepatitis_B']+df['Diphtheria'])/(df['Percentage_Expenditure']+df['Total_Expenditure']))

Schooling is an important factor for knowledge about hygiene & financial independence - Here values are too much skewed so 1st we apply log transformation then calculate Yearly costing for school . By doing so we can estimate a country with low GDP can afford schooling or not<br>

Don't replace 0 as log(1)=0 , so it's another reason to apply log transform 1st to avoid zero division error <br>

In [1086]:
#df['Schooling']=df['Schooling'].replace(0,-1)
df['Yearly_Cost_Schooling']=np.log(df['Percentage_Expenditure']+df['Total_Expenditure'])/np.log(df['Schooling'])

Here both feature shows almost same property but distributon is a bit different , if we take difference we loose the variance in a certain range so we scale down (using log transformation) then take avg. to preserve the variance in mostly Under_Weight children range - **Under_weight child has more risk of dying than teenagers**<br>

In [1087]:
df['Under_Weight']=(np.log(df['Thinness_1-19_Years'])+np.log(df['Thinness_5-9_Years']))/2

The mortality rates of normal-weight individuals who were formerly overweight or obese were 47.48 and 66.67 per 1000 person-years, respectively. Therefore, people who converted from overweight or obese to normal weight at baseline raised the mortality rate of the normal-weight category to 33.81 per 1000 person-years.

BMI value > 30 = Obesity one primary cause of taking 7g of pure-alcohol daily i.e 2.5 Liter yearly(per 100 person above 15)

In [1088]:
df['Changed_Life_Expectancy']=df['Life_Expectancy']
df.loc[(df['BMI']>30) & (df['Alcohol']>2.5),'Changed_Life_Expectancy']=df['Life_Expectancy']*(1-0.34)

In [1089]:
df['Changed_Life_Expectancy']=np.log(df['Changed_Life_Expectancy'])

In [1090]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

In [1091]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score,cross_val_predict
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

In [1092]:
df_new=df
df_new.drop(['Country','Year','Status'],axis=1,inplace=True)

In [1093]:
#converting both of them to integer as float can be large value for Linear Regression
X=df_new.iloc[:,1:].astype('int')
Y=df_new.iloc[:,0].astype('int')

In [1094]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=2)

In [1095]:
scaler=RobustScaler()
#X_train_scaled=scaler.fit_transform(X_train) 
# let's do fit() & transform() separately for X_train
X_train_fit=scaler.fit(X_train)
X_train_scaled=scaler.transform(X_train)
# transform X_test
X_test_scaled=scaler.transform(X_test) 
#use orginal variable for convenience
X_train=X_train_scaled
X_test=X_test_scaled

In [1096]:
df_tmp=pd.DataFrame(X_train,columns=X.columns)
stats = df_tmp.describe()
stats.loc['median'] = df_tmp.median().tolist()
stats.loc['skew'] = df_tmp.skew().tolist()
stats.loc['kurt'] = df_tmp.kurtosis().tolist()
stats_df=pd.DataFrame(stats).reindex(['count','mean','std','median','50%','skew','kurt','min','max','25%','75%'])
stats_df 

In [1097]:
lr = LinearRegression()
lr = lr.fit(X_train,Y_train)

In [1098]:
#model evaluation for training set
Y_train_predict = lr.predict(X_train)
rmse = (np.sqrt(mean_squared_error(Y_train,Y_train_predict)))
r2 = r2_score(Y_train, Y_train_predict)
print("The model performance for training set")
print(f"Train Error(rmse) : {rmse}")
print(f"Train r2 score : {r2}")

In [1099]:
#model evaluation for test set
Y_test_predict = lr.predict(X_test)
rmse = (np.sqrt(mean_squared_error(Y_test,Y_test_predict)))
r2 = r2_score(Y_test,Y_test_predict)
print("The model performance for test set")
print(f"Train Error(rmse) : {rmse}")
print(f"Test r2 score : {r2}")

R^2 - represents how much variance of the data is explained by the model, the R2=0.90 means that 0.10 of the variance can not explain by the model, the logical case when R^2=1 the model completely fit and explained all variance <br>

Cross-validation - primarily used in applied machine learning to estimate the skill of a machine learning model on unseen data <br>

we can't look at test data so we use part of train data to validate our model or in other words to 
simulate test set<br>

mean squared error or mean squared deviation of an estimator -  measures the average of the squares of the errors—that is, the average squared difference between the estimated values and the actual value. MSE is a risk function, corresponding to the expected value of the squared error loss. <br>

In [1100]:
folds=10
kf = KFold(shuffle=True,random_state=1,n_splits=folds)
cross_validation=cross_val_score(lr,X_train,Y_train, cv=kf,scoring='r2')
print(f"Cross-validated scores with splits = {folds} : {cross_validation}")
print(f"Mean value of {folds} cross validation score {np.mean(cross_validation)}")

In [1101]:
predictions = cross_val_predict(lr,X_train,Y_train,cv=kf)
fig, ax = plt.subplots()
ax.scatter(Y_train, predictions,)
ax.plot([Y_train.min(), Y_train.max()], [Y_train.min(), Y_train.max()], "k--")
ax.set_xlabel("Measured Life_Expectancy")
ax.set_ylabel("Predicted Life_Expectancy")
plt.title("Measured vs Predicted Life_Expectancy")
plt.show()

In [1102]:
print(f"Mean Square Error : {mean_squared_error(Y_train, predictions)}")
print(f"r2 Score(Cross-Predicted Accuracy) : {r2_score(Y_train, predictions)}")

So if we let automated feature selection do the work it's giving 82% accuracy , let's try our hand engineered features & lets see if we can improve or not

In [1103]:
# Y_pred=pd.DataFrame(predictions,columns=['Y_pred'])
# Y_pred=scaler.inverse_transform(Y_pred['Y_pred'])
# Y_pred

In [1104]:
# plt.scatter(df_new['Life_Expectancy'],predictions)
# plt.xlabel('Life_Expectancy orginal')
# plt.ylabel('Life_Expectancy predicted')
# plt.title('Orginal vs Predicted')

In [1105]:
# automated feature selection - using random forest
random_forest=RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train,Y_train)
df_tmp=pd.DataFrame(X_train,columns=X.columns)
importances=pd.DataFrame({'feature':df_tmp.columns ,'importance':random_forest.feature_importances_})
importances.sort_values('importance',ascending=False).set_index('feature')
importances

Now let's try with our engineered features , so dropping other features

In [1106]:
#drop unnecessary column
df_handpicked=df
df_handpicked.drop(['Adult_Mortality','Infant_Deaths',
        'Percentage_Expenditure', 'Hepatitis_B',
       'Measles','Under-Five_Deaths', 'Polio', 'Total_Expenditure',
       'Diphtheria', 'HIV_AIDS', 'GDP', 'Population','Thinness_1-19_Years',
       'Thinness_5-9_Years','Schooling'],axis=1,inplace=True)

In [1107]:
# using IQR method to find outliers & replacing ihem with lower_range , upper_range 
#since our dataset is small
def outlier_imputation(data):
    for col in data.columns.values:
        sorted(data[col])
        q1,q3=np.percentile(data,[25,75])
        iqr=q3-q1
        lower_range=q1-(1.5*iqr)
        upper_range=q3+(1.5*iqr)
        data[data[col]<lower_range]=lower_range
        data[data[col]>upper_range]=upper_range
    return data

In [1108]:
#apply outlier imputation
df_handpicked=outlier_imputation(df_handpicked[['Alcohol','BMI','ICR','Disease_Death','Mortality_Rate', 
                          'Vaccination_Immunity', 'Yearly_Cost_Schooling','Under_Weight', 
                          'Changed_Life_Expectancy']])

In [1109]:
col=['Alcohol','BMI','ICR', 'Disease_Death','Mortality_Rate', 'Vaccination_Immunity', 
     'Yearly_Cost_Schooling','Under_Weight', 'Changed_Life_Expectancy']
dic={}
for i in range(len(col)):
    dic[col[i]]=i+1
# Detect outliers in each variable using box plots.
plt.figure(figsize=(20,30))

for key,val in dic.items():
    plt.subplot(5,5,val)
    sns.boxplot(x=df_handpicked[key],data=df_handpicked,boxprops=dict(alpha=0.3))
    plt.xlabel(key)
    plt.title(key+' Distribution')

plt.show()

In [1110]:
#converting both of them to integer as float can be large value for Linear Regression
X=df_handpicked.iloc[:,1:].astype('int')
Y=df_handpicked.iloc[:,0].astype('int')

In [1111]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=2)

In [1112]:
scaler=RobustScaler()
#X_train_scaled=scaler.fit_transform(X_train) 
# let's do fit() & transform() separately for X_train
X_train_fit=scaler.fit(X_train)
X_train_scaled=scaler.transform(X_train)
# transform X_test
X_test_scaled=scaler.transform(X_test) 
#use orginal variable for convenience
X_train=X_train_scaled
X_test=X_test_scaled

In [1113]:
df_tmp=pd.DataFrame(X_train)
stats = df_tmp.describe()
stats.loc['median'] = df_tmp.median().tolist()
stats.loc['skew'] = df_tmp.skew().tolist()
stats.loc['kurt'] = df_tmp.kurtosis().tolist()
stats_df=pd.DataFrame(stats).reindex(['count','mean','std','median','50%','skew','kurt','min','max','25%','75%'])
stats_df 

In [1114]:
lr = LinearRegression()
lr = lr.fit(X_train,Y_train)

In [1115]:
#model evaluation for training set
Y_train_predict = lr.predict(X_train)
rmse = (np.sqrt(mean_squared_error(Y_train,Y_train_predict)))
r2 = r2_score(Y_train, Y_train_predict)
print("The model performance for training set")
print(f"Train Error(rmse) : {rmse}")
print(f"Train r2 score : {r2}")

In [1116]:
#model evaluation for test set
Y_test_predict = lr.predict(X_test)
rmse = (np.sqrt(mean_squared_error(Y_test,Y_test_predict)))
r2 = r2_score(Y_test,Y_test_predict)
print("The model performance for test set")
print(f"Train Error(rmse) : {rmse}")
print(f"Test r2 score : {r2}")

high mean and low standard deviation of your quality measure would mean the modeling technique is doing well. 

In [1117]:
folds=10
kf = KFold(shuffle=True,random_state=1,n_splits=folds)
cross_validation=cross_val_score(lr,X_train,Y_train, cv=kf,scoring='r2')
print(f"Cross-validated scores with splits = {folds} : {cross_validation}")
print(f"Mean value of {folds} cross validation score {np.mean(cross_validation)}")

In [1118]:
#predictions = cross_val_predict(lr,X,Y,cv=kf)
predictions = cross_val_predict(lr,X_train,Y_train,cv=kf)
fig, ax = plt.subplots()
ax.scatter(Y_train, predictions)
ax.plot([Y_train.min(), Y_train.max()], [Y_train.min(), Y_train.max()], "k--")
ax.set_xlabel("Measured Life_Expectancy")
ax.set_ylabel("Predicted Life_Expectancy")
plt.title("Measured vs Predicted Life_Expectancy")
plt.show()

In [1119]:
print(f"Mean Square Error : {mean_squared_error(Y_train, predictions)}")
print(f"r2 Score(Cross-Predicted Accuracy) : {r2_score(Y_train, predictions)}")

In [1120]:
# using logistice regressor object to calculate correlation of features
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
# calculate correlation coefficient
coeff_df = pd.DataFrame(df_handpicked.columns.delete(0)) # drop survived 
coeff_df.columns = ["Feature"]
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by="Correlation", ascending=False)
coeff_df

# Who is the winner ?? 

So our hand engineered featurs are giving much more accuracy than just random forest feature importance method as we can see our hand engineered <br>

**Automated feature selection accuracy (r2 score) - 84%** <br>
**Hand engineered feature selection accuracy (r2 score) - 98%** <br>

**Point of concern - As we have applied different kinds of Gaussian transformation the sprade of data points have shrinked,though it helped here to eliminate certain overlapping features but is it a true representation of population that is a big question** <br>


We can try using other type of regression with regularization to check if we can get an even better accuracy 
# And here the winner is - Hand Engineered Feature , But is it the true winner or there is something wrong about our approach